In [91]:
# Import machine learning library
using Flux
using Flux.Tracker
using Flux: @epochs

In [60]:
# Define our model
model = Chain(
    Dense(4, 4, relu),
    Dense(4, 4, relu),
    Dense(4, 3, relu),
    Dense(3, 1, relu)
)

# Squared loss since target values are in a range
loss(x, y) = sum((model(x) .- y).^2)
optimizer = Flux.Optimise.ADAM(params(model));

In [66]:
# Load our data (inputs and labels)
fakedata = [([1; 2; 3; 4] / 10, 3 / 10), ([2; 1; 4; 3] / 10, 2 / 10), ([4; 5; 6; 7] / 10, 1 / 10), ([7; 3; 2; 1] / 10, 3 / 10)]


4-element Array{Tuple{Array{Float64,1},Float64},1}:
 ([0.1, 0.2, 0.3, 0.4], 0.3)
 ([0.2, 0.1, 0.4, 0.3], 0.2)
 ([0.4, 0.5, 0.6, 0.7], 0.1)
 ([0.7, 0.3, 0.2, 0.1], 0.3)

In [110]:
test_x, test_y = fakedata[1]
observer() = @show(loss(test_x, test_y))
cb = Flux.throttle(observer, 1, leading=false, trailing=true)
for i in 1:10000
    Flux.train!(loss, fakedata, optimizer);
    cb()
end

loss(test_x, test_y) = 0.0 (tracked)
loss(test_x, test_y) = 0.0 (tracked)
loss(test_x, test_y) = 0.0 (tracked)
loss(test_x, test_y) = 0.0 (tracked)
loss(test_x, test_y) = 0.0 (tracked)
loss(test_x, test_y) = 0.0 (tracked)


In [113]:
model([1; 7; 3; 5] / 10)

Tracked 1-element Array{Float64,1}:
 0.0983554

In [102]:
?Flux.throttle

Returns a function that when invoked, will only be triggered at most once during `timeout` seconds. Normally, the throttled function will run as much as it can, without ever going more than once per `wait` duration; but if you'd like to disable the execution on the leading edge, pass `leading=false`. To enable execution on the trailing edge, ditto.


In [ ]:
# Create our training data
using ExcelFiles, CSVFiles, DataFrames

function loadRankings(filename="data/team-rankings.csv")
    team_rankings = load(filename) |> DataFrame;
    rankings = Dict()
    max_points = maximum(team_rankings[:Points])
    for i in 1:length(team_rankings[:Team])
        row = team_rankings[i, [:Team, :Points, :Bias]]
        rankings[row[:Team][1]] = (row[:Points][1] / max_points, row[:Bias][1])
    end
    rankings
end

function loadGames(filename="data/group-phase.xlsx")
    group_phase = load(filename, "Sheet1") |> DataFrame;
    group_phase
end

function createTrainingData()
    rankings = loadRankings()
    games = loadGames()
    # return (x, y) tuples of input teams and output scores
    [(input, output), (input, output)]
end